In [17]:
import natsort
from natsort import natsorted
import os
import pandas as pd
import re

In [32]:
time_conv_dict = {'as': 1e-18,
                  'attosec': 1e-18,
                  'attosecond': 1e-18,
                  'attoseconds': 1e-18,
                  'fs': 1e-15,
                  'femtosec': 1e-15,
                  'femtosecond': 1e-15,
                  'femtoseconds': 1e-15,
                  'ps': 1e-12,
                  'picosec': 1e-12,
                  'picosecond': 1e-12,
                  'picoseconds': 1e-12,
                  'ns': 1e-9,
                  'nanosec': 1e-9,
                  'nanosecond': 1e-9,
                  'nanoseconds': 1e-9,
                  'us': 1e-6,
                  'microsec': 1e-6,
                  'microsecond': 1e-6,
                  'microseconds': 1e-6,
                  'ms': 1e-3,
                  'millisec': 1e-3,
                  'millisecond': 1e-3,
                  'milliseconds': 1e-3,
                  's': 1.0,
                  'sec': 1.0,
                  'second': 1.0,
                  'seconds': 1.0,
                  'm': 60.0,
                  'min': 60.0,
                  'minute': 60.0,
                  'minutes': 60.0,
                  'h': 3600.0,
                  'hour': 3600.0,
                  'hours': 3600.0,
                  'd': 86400.0,
                  'day': 86400.0,
                  'days': 86400.0,
                  'w': 86400.0*7.0,
                  'week': 86400.0*7.0,
                  'weeks': 86400.0*7.0,
                  'y': 86400.0*365.25,
                  'year': 86400.0*365.25,
                  'years': 86400.0*365.25,
                  'c': 86400.0*365.25*100,
                  'century': 86400.0*365.25*100,
                  'centuries': 86400.0*365.25*100,
                  }

In [33]:
_valexp = re.compile('([0-9.]*)([Ee][+-]?\d*)')
_val = re.compile('(\d*)[.](\d*)')
_specialval = re.compile("([0-9. ]*)[+]([A-Z])")
_specialval2 = re.compile("([A-Z]*)[+]([0-9.]*)")
_errpm = re.compile('[+](\d*)[-](\d*)')
_err = re.compile('[ ]*(\d*)')
_base = '([ \d]{3}[ A-Za-z]{2})'
_ident = re.compile(_base + '    (.{30})(.{26})(.{7})(.{6})')
_g = re.compile(_base + '  G (.{10})(.{2})(.{8})(.{2}).{24}(.{7})(.{2})(.{10})'
                + '(.{2})')
_gc = re.compile(_base + '[0-9A-Za-z] [GE] (.{70})')
_beta = re.compile(_base + '  B (.{10})(.{2})(.{8})(.{2}).{10}(.{8})(.{6})')
_betac = re.compile(_base + '[0-9A-Za-z] ([BE]) (.{70})')
_ec = re.compile(_base + '  E (.{10})(.{2})(.{8})(.{2})'
                 + '(.{8})(.{2})(.{8})(.{6})(.{10})(.{2})')
_p = re.compile(_base + '  P (.{10})(.{2})(.{18})(.{10})'
                + '(.{6}).{9}(.{10})(.{2})(.{4})')
_norm = re.compile(_base + '  N (.{10})(.{2})(.{8})(.{2})(.{8})(.{2})(.{8})'
                   + '(.{6})(.{7})(.{2})')
_normp = re.compile(_base +
                    ' PN (.{10})(.{2})(.{8})(.{2})(.{8})(.{2})(.{7})(.{2})')
_q = re.compile(_base + '  Q (.{10})(.{2})(.{8})(.{2})'
                + '(.{8})(.{2})(.{8})(.{6})')
_alpha = re.compile(_base + '  A (.{10})(.{2})(.{8})(.{2})(.{8})(.{2})')
_dp = re.compile(_base + '  D(.{1})(.{10})(.{2})(.{8})(.{2})(.{8})(.{10})'
                 + '(.{6})')
_decays = ['B-', 'B+A', 'EC', 'B-A', 'B+', 'B+P', 'B-N', 'ECP', 'EC2P', 'N',
           '2N', 'IT', 'B+2P', 'B-2N', 'B+3P', 'ECA', 'P', '2P', '2B-', 'SF',
           'A', '2B+', '2EC', '14C']
_level_regex = re.compile(_base + '  L (.{10})(.{2})(.{18})(.{10})(.{6})'
                          + '(.{9})(.{10})(.{2})(.{1})([ M])([ 1-9])')
_level_cont_regex = re.compile('([ \d]{3}[ A-Za-z]{2})[0-9A-Za-z] L (.*)')

def levels(filename, levellist=None):
    badlist = ["ecsf", "34si", "|b{+-}fission", "{+24}ne",
               "{+22}ne", "24ne", "b-f", "{+20}o", "2|e", "b++ec",
               "ecp+ec2p", "ecf", "mg", "ne", "{+20}ne", "{+25}ne",
               "{+28}mg", "sf(+ec+b+)"]
    special = ""
    if levellist is None:
        levellist = []
    if isinstance(filename, str):
        with open(filename, 'r') as f:
            dat = f.read()
    else:
        dat = filename.read()
    datasets = dat.split(80 * " " + "\n")[0:-1]
    for dataset in datasets:
        lines = dataset.splitlines()
        ident = re.match(_ident, lines[0])
        if ident is None:
            continue
        if 'ADOPTED LEVELS' in ident.group(2):
            leveln = 0
            brs = {}
            level_found = False
            for line in lines:
                level_l = _level_regex.match(line)
                if level_l is not None:
                    if len(brs) > 0:
                        for key, val in brs.items():
                            goodkey = True
                            keystrip = key.replace("%", "").lower()
                            for item in badlist:
                                if keystrip == item:
                                    goodkey = False
                            if goodkey is True:
                                rx = rxname.id(keystrip)
                                branch_percent = float(val.split("(")[0])
                                levellist.append((nuc_id, rx, half_lifev,
                                                  level, branch_percent,
                                                  state, special))
                    if level_found is True:
                        levellist.append((nuc_id, 0, half_lifev, level, 0.0,
                                          state, special))
                    brs = {}
                    level, half_lifev, from_nuc, state, special = \
                        _parse_level_record(level_l)
                    if from_nuc is not None:
                        nuc_id = from_nuc + leveln
                        leveln += 1
                        level_found = True
                    else:
                        level_found = False
                    continue
                levelc = _level_cont_regex.match(line)
                if levelc is not None:
                    brs.update(_parse_level_continuation_record(levelc))
                    continue
            if len(brs) > 0:
                for key, val in brs.items():
                    goodkey = True
                    keystrip = key.replace("%", "").lower()
                    for item in badlist:
                        if keystrip == item:
                            goodkey = False
                    if goodkey is True:
                        rx = rxname.id(keystrip)
                        branch_percent = float(val.split("(")[0])
                        levellist.append((nuc_id, rx, half_lifev, level,
                                          branch_percent, state, special))
            if level_found is True:
                levellist.append((nuc_id, 0, half_lifev, level, 0.0, state,
                                  special))
#     _adjust_ge100_branches(levellist)
#     _adjust_metastables(levellist)
#     _adjust_half_lives(levellist)
    return levellist

In [34]:
levels("./ENSDF_Data/ensdf.001")

C:\Users\Pedro\Anaconda3\envs\data_mining_2\lib\site-packages\ipykernel_launcher.py:67: UserWarning: Neutron data not supported!


NameError: name 'rxname' is not defined